<a href="https://colab.research.google.com/github/fliroajr/personal-labs/blob/main/dsa_data_engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Demonstração Prática 4 - Criação de Pipeline de Extração, Limpeza, Transformação e Enriquecimento de Dados

# V1

In [46]:
# importação das bibliotecas
import urllib.request as url
import csv
import sqlite3

In [47]:
# cria um banco de dados SQLite
conn = sqlite3.connect('dsadb.db')

In [48]:
# Cria uma tabela para armazenar os dados de produção de alimentos
conn.execute('''CREATE TABLE IF NOT EXISTS producao (
                produto TEXT,
                quantidade INTEGER,
                preco_medio REAL,
                receita_total REAL
            )''')

# Grava e fecha a conexão
conn.commit()
conn.close()

In [49]:
# Download do arquivo csv do repo github
url.urlretrieve("https://github.com/fliroajr/personal-labs/raw/main/producao_alimentos.csv", "producao_alimentos.csv")

('producao_alimentos.csv', <http.client.HTTPMessage at 0x7b2ad0c63340>)

In [50]:
with open('/content/producao_alimentos.csv', 'r') as file:

    # Cria um leitor de CSV para ler o arquivo
    reader = csv.reader(file)

    # Pula a primeira linha, que contém os cabeçalhos das colunas
    next(reader)

    # Conecta ao banco de dados
    conn = sqlite3.connect('dsadb.db')

    # Insere cada linha do arquivo na tabela do banco de dados
    for row in reader:
        conn.execute('INSERT INTO producao (produto, quantidade, preco_medio, receita_total) VALUES (?, ?, ?, ?)', row)

    conn.commit()
    conn.close()

print("Job Concluído com Sucesso!")

Job Concluído com Sucesso!


# v2

Regra de negócio: Carregar somente registros com quantidade produzida superior a 10.

In [51]:
with open('/content/producao_alimentos.csv', 'r') as file:

    # Cria um leitor de CSV para ler o arquivo
    reader = csv.reader(file)

    # Pula a primeira linha, que contém os cabeçalhos das colunas
    next(reader)

    # Conecta ao banco de dados
    conn = sqlite3.connect('dsadb.db')

    # Dropa e recria a tabela producao se já existe
    conn.execute('''DROP TABLE IF EXISTS producao;''')

    conn.execute('''CREATE TABLE IF NOT EXISTS producao (
                produto TEXT,
                quantidade INTEGER,
                preco_medio REAL,
                receita_total REAL
            );''')

    # Insere cada linha do arquivo na tabela do banco de dados
    for row in reader:
      if int(row[1]) > 10:
        conn.execute('INSERT INTO producao (produto, quantidade, preco_medio, receita_total) VALUES (?, ?, ?, ?)', row)

    conn.commit()
    conn.close()

print("Job Concluído com Sucesso!")

Job Concluído com Sucesso!


# V3

Regra de negócio: Remover o caracter "ponto" na última coluna do arquivo para evitar que o número seja truncado.

In [52]:
# Função para remover o ponto nos dados da última coluna
def remove_ponto(valor):
    return int(valor.replace('.', ''))

In [53]:
with open('/content/producao_alimentos.csv', 'r') as file:

    # Cria um leitor de CSV para ler o arquivo
    reader = csv.reader(file)

    # Pula a primeira linha, que contém os cabeçalhos das colunas
    next(reader)

    # Conecta ao banco de dados
    conn = sqlite3.connect('dsadb.db')

    # Dropa e recria a tabela producao se já existe
    conn.execute('''DROP TABLE IF EXISTS producao;''')

    conn.execute('''CREATE TABLE IF NOT EXISTS producao (
                produto TEXT,
                quantidade INTEGER,
                preco_medio REAL,
                receita_total REAL
            );''')

    # Insere cada linha do arquivo na tabela do banco de dados
    for row in reader:
      if int(row[1]) > 10:

        # Remove ponto do valor da ultima coluna e converte em inteiro
        row[3] = remove_ponto(row[3])

        # Insere dados no banco
        conn.execute('INSERT INTO producao (produto, quantidade, preco_medio, receita_total) VALUES (?, ?, ?, ?)', row)

    conn.commit()
    conn.close()

print("Job Concluído com Sucesso!")

Job Concluído com Sucesso!


# V4

Enriquecer os dados adicionando no destino uma coluna com a margem de lucro de cada produto

In [54]:
with open('/content/producao_alimentos.csv', 'r') as file:

    # Cria um leitor de CSV para ler o arquivo
    reader = csv.reader(file)

    # Pula a primeira linha, que contém os cabeçalhos das colunas
    next(reader)

    # Conecta ao banco de dados
    conn = sqlite3.connect('dsadb.db')

    # Dropa e recria a tabela producao se já existe
    conn.execute('''DROP TABLE IF EXISTS producao;''')

    # Cria a tabela com uma nova coluna atendendo à necessidade da regra de negócio
    conn.execute('''CREATE TABLE IF NOT EXISTS producao (
                produto TEXT,
                quantidade INTEGER,
                preco_medio REAL,
                receita_total REAL,
                margem_lucro REAL
            );''')

    # Insere cada linha do arquivo na tabela do banco de dados
    for row in reader:
      if int(row[1]) > 10:

        # Remove ponto do valor da ultima coluna e converte em inteiro
        row[3] = remove_ponto(row[3])

        # Calcula a margem de lucro bruta com base no valor médio de venda e na receita total
        margem_lucro = (row[3] / float(row[1])) - float(row[2])

        # # Insere a linha com a nova coluna 'margem_lucro' na tabela do banco de dados
        conn.execute('INSERT INTO producao (produto, quantidade, preco_medio, receita_total, margem_lucro) VALUES (?, ?, ?, ?, ?)', (row[0], row[1], row[2], row[3], margem_lucro))

    conn.commit()
    conn.close()

print("Job Concluído com Sucesso!")

Job Concluído com Sucesso!


# V5

Formatando as casas decimais utilizando função 'round' para arredondar

In [57]:
with open('/content/producao_alimentos.csv', 'r') as file:

    # Cria um leitor de CSV para ler o arquivo
    reader = csv.reader(file)

    # Pula a primeira linha, que contém os cabeçalhos das colunas
    next(reader)

    # Conecta ao banco de dados
    conn = sqlite3.connect('dsadb.db')

    # Dropa e recria a tabela producao se já existe
    conn.execute('''DROP TABLE IF EXISTS producao;''')

    # Cria a tabela com uma nova coluna atendendo à necessidade da regra de negócio
    conn.execute('''CREATE TABLE IF NOT EXISTS producao (
                produto TEXT,
                quantidade INTEGER,
                preco_medio REAL,
                receita_total REAL,
                margem_lucro REAL
            );''')

    # Insere cada linha do arquivo na tabela do banco de dados
    for row in reader:
      if int(row[1]) > 10:

        # Remove ponto do valor da ultima coluna e converte em inteiro
        row[3] = remove_ponto(row[3])

        # Calcula a margem de lucro bruta com base no valor médio de venda e na receita total
        margem_lucro = round((row[3] / float(row[1])) - float(row[2]), 2)

        # # Insere a linha com a nova coluna 'margem_lucro' na tabela do banco de dados
        conn.execute('INSERT INTO producao (produto, quantidade, preco_medio, receita_total, margem_lucro) VALUES (?, ?, ?, ?, ?)', (row[0], row[1], row[2], row[3], margem_lucro))

    conn.commit()
    conn.close()

print("Job Concluído com Sucesso!")

Job Concluído com Sucesso!
